# 5.4 自定义层
可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
如用于处理图像、文本、序列数据和执行动态规划的层。 
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。 在这些情况下，必须构建自定义层。

+ 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。

+ 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。

+ 层可以有局部参数，这些参数可以通过内置函数创建。

## 5.4.1 不带参数的层

In [6]:
# 从输入当中减去均值的层
import torch
import torch.nn.functional as F 
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [7]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [9]:
# 测试浮点数 最后结果的mean应该为0 但是精度问题会导致最后是一个-inf
# 将CenteredLayer组合到更大的网络当中
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())
Y = net(torch.rand(2,8))
Y.mean()

tensor(6.9849e-09, grad_fn=<MeanBackward0>)

## 5.4.2 带参数的层

In [10]:
# 自定义版本的全连接层 W 和 bias
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        # 修正线性单元
        return F.relu(linear)
    

In [14]:
# 实例化MyLinear类并访问其参数
linear = MyLinear(5,3)
# torch.nn.parameter.Parameter类型
print((linear.weight)) 
# tensor类型
print((linear.weight.data))


Parameter containing:
tensor([[ 1.3275,  0.4442, -0.9269],
        [ 0.1498,  0.1826, -0.1862],
        [-2.1918,  2.5479, -1.0744],
        [ 1.0381, -0.6894,  0.7876],
        [-0.0142, -0.5855,  0.2298]], requires_grad=True)
tensor([[ 1.3275,  0.4442, -0.9269],
        [ 0.1498,  0.1826, -0.1862],
        [-2.1918,  2.5479, -1.0744],
        [ 1.0381, -0.6894,  0.7876],
        [-0.0142, -0.5855,  0.2298]])


In [15]:
# 使用自定义层直接执行前向传播计算
linear(torch.rand(2,5))

tensor([[0.0000, 0.3891, 0.0000],
        [0.7395, 0.0000, 0.3609]])

In [16]:
# 使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.3020],
        [6.2328]])

$y_k= {\textstyle \sum_{i}^{j}}W_{i,j,k}x_ix_j$

In [17]:
import torch
import torch.nn as nn

class CustomReductionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CustomReductionLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(in_channels, in_channels, out_channels))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        # Expand dimensions for element-wise multiplication
        #  维度2扩展 （N,in_channels,1）
        x_expanded = x.unsqueeze(2)
        # 维度1扩展 (N,1,in_channels)
        x_expanded_transpose = x.unsqueeze(1)
        
        # Element-wise multiplication and summation
        y = torch.sum(x_expanded * x_expanded_transpose * self.weight, dim=(0, 1))
        
        return y


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 2